In [153]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.coordinates import SkyCoord
import warnings
import pickle
import sys
import copy
warnings.filterwarnings('ignore')
def mass_radius(R):
    if R <= 1.23*u.earthRad:
        return(0.9718 * R.to('earthRad').value ** 3.58 * u.earthMass)
    elif R > 1.23*u.earthRad and R < 14.26*u.earthRad:
        return(1.436 * R.to('earthRad').value ** 1.7 * u.earthMass)
    else:
        return(0* u.earthMass)

Define observing constraints: 

In [154]:
#Equilibrium temperature, radius and Gaia mag.
teq_min = 0 * u.K
teq_max = 1500 * u.K
rad_min = 2.0 * u.R_earth
rad_max = 5.0 * u.R_earth
gaia_mag_limit = 15

JWST_mode = 'NIRSpec Prism'
#Available modes:
#'NIRSpec G140M','NIRSpec G235M','NIRSpec G395M','NIRSpec Prism','NIRISS SOSS','NIRCam F322W2','NIRCam F444W','MIRI LRS')


Download NASA Exoplanet Archive table from the web via astroquery, fix up the semi major axis using the stellar mass (sigh...), select the part of the population that is transiting and save it to a pickle file for running Pandexo simulations with Pandexo.py. Any planets that do not have a transit duration listed will get an approximation based on an impact parameter of 0.6. WARNING: Do not run this script while Pandexo.py is running!

In [155]:
table = NasaExoplanetArchive.get_confirmed_planets_table(all_columns=True)
transiting = table[(table['pl_tranflag'].data).astype(bool)]

transiting['sma'] = transiting['pl_orbsmax']*1.0
a_computed = ((const.G*transiting['st_mass']*transiting['pl_orbper']**2 /(4.0*np.pi**2))**(1/3)).to('AU')
transiting['sma'][transiting['sma']==0.0] = a_computed[transiting['sma']==0.0]
transiting['duration_predicted']=(transiting['pl_orbper']/np.pi * np.arcsin(np.sqrt((transiting['st_rad']+transiting['pl_radj'])**2 - (0.6*transiting['st_rad'])**2)/transiting['sma'])/u.rad).to('h')
#This is an analytical approximation of the transit duration sourced from https://www.paulanthonywilson.com/exoplanets/exoplanet-detection-techniques/the-exoplanet-transit-method/
#assuming an average impact parameter of 0.6. The predicted duration will be used by Pandexo if the duration is missing from the archive.
#So any planet with an SNR estimate without a listed duration will have this approximation applied to it.
pickle.dump( transiting, open( "table.p", "wb" ) )

After this line, you should run the accompanying script Pandexo.py to add Pandexo simulations of the SNR and observation time (minus overhead) as columns to the transiting planets table. This table is then read in again in the following line:

In [156]:
transiting = pickle.load(open("table_pandexo.p", "rb" ))

Compute equilibrium temperatures for zero albedo, the (relative) scale height, approximate the mass using an RM relation and approximate the transit duration for use with PandExo for those planets that don't have one.

In [157]:
rp = transiting['pl_radj']
equilibrium_temperature = (transiting['st_teff'] * np.sqrt(transiting['st_rad'] / 2 / transiting['sma'])).decompose()
g = transiting['gaia_gmag'].quantity
transiting['teq'] = equilibrium_temperature

mean_molecular_mass = 6 * u.u#Assuming a default mean molecular weight. Unknown for almost all targets.
transiting['mass_predicted']=[mass_radius(x) for x in rp]

# gp = const.G * 10.0*u.earthMass / rp**2#Assuming a mass of 10Me
gp = const.G * transiting['mass_predicted'] / rp**2#Assuming a mass from the Chen et al. M-R. 
#Most of our targets have no masses. For the ones that do, we'd need to take the difference in H into account by eye.
transiting['H'] = ((const.k_B * equilibrium_temperature) / (mean_molecular_mass * gp)).to(u.km)
transiting['signal'] = (2*rp*transiting['H']/transiting['st_rad']**2).decompose()
transiting[JWST_mode+'_error'][transiting[JWST_mode+'_error']<0]=np.nan#Set planets that were skipped by Pandexo to NaN.
transiting['Kempton_metric'] = rp**3 / transiting['mass_predicted'] *equilibrium_temperature/transiting['st_rad']**2 / 10**(transiting['st_j']/5.0)

<br/>
Apply the selection constraints and prepare for output:

In [158]:
temp_constraints = (equilibrium_temperature < teq_max) & (equilibrium_temperature > teq_min)
rad_constraints = (rp < rad_max) & (rp > rad_min)
gmag_constaints = (g < gaia_mag_limit)
# transiting[['pl_name','pl_trandur','duration_predicted']].show_in_notebook()
targets = transiting[rad_constraints & temp_constraints & gmag_constaints]#This is the table that will be plotted.

targets.sort('st_j')
targets['r_earth'] = targets['pl_radj'].to(u.R_earth)
targets['Teq']=np.round(targets['teq'],0)
targets['Rp']=np.round(targets['r_earth'],2)
targets['H (predicted)']=np.round(targets['H'],0)
targets['Orbital Period']=np.round(targets['pl_orbper'],2)
targets['DEC']=np.round(targets['dec'],1)
targets['Dur']=np.round(targets['pl_trandur']*24.0,1)*u.h
targets['Dur'][targets['Dur']==0.0]=np.nan
targets['Dur (predicted)']=np.round(targets['duration_predicted'],1)
targets['H_ppm']=np.round(1e6*targets['signal'],2)
targets['JWST_ppm']=np.round(1e6*targets[JWST_mode+'_error'],1)
targets['SNR_H']=np.round(targets['signal']/targets[JWST_mode+'_error'],4)
targets['TSM (J)'] = np.round(targets['Kempton_metric'].value*100,1)
targets['T_eff']=np.round(targets['st_teff'],0)
targets['Mass (measured)']=np.round(targets['pl_bmasse'],2)*u.earthMass
targets['Mass (predicted)']=np.round(targets['mass_predicted'],2)
targets[['pl_name', 'st_j', 'Teq', 'Rp', 'JWST_ppm' ,'Mass (measured)','Mass (predicted)','Orbital Period','H (predicted)','H_ppm','SNR_H','TSM (J)','Dur','Dur (predicted)','DEC','T_eff','st_spstr']].show_in_notebook()

idx,pl_name,st_j,Teq,Rp,JWST_ppm,Mass (measured),Mass (predicted),Orbital Period,H (predicted),H_ppm,SNR_H,TSM (J),Dur,Dur (predicted),DEC,T_eff,st_spstr
,,,K,earthRad,,earthMass,earthMass,d,km,,,,h,h,deg,K,
0,pi Men c,4.869,1167.0,2.04,14.8,4.82,4.83,6.27,142.0,6.33,0.4278,12.8,3.0,2.9,-80.5,6037.0,G0 V
1,HR 858 d,5.473,1068.0,2.16,18.3,0.0,5.33,11.23,133.0,4.4,0.2408,6.7,3.4,4.2,-30.8,6201.0,F6 V
2,GJ 143 b,6.081,425.0,2.61,21.7,22.7,7.34,35.61,56.0,8.07,0.3713,9.3,3.2,3.8,-63.5,4640.0,K4.5
3,HD 97658 b,6.203,739.0,2.35,27.8,9.53,6.15,9.49,94.0,10.66,0.3837,11.7,nan,2.5,25.7,5175.0,K1
4,K2-167 b,7.202,1275.0,2.82,30.8,0.0,8.39,9.98,172.0,3.81,0.1237,2.6,nan,5.8,-18.0,5908.0,F7 V
5,HD 3167 c,7.548,579.0,2.86,41.2,0.0,8.56,29.84,78.0,7.78,0.1887,4.6,nan,4.2,4.4,5528.0,K0 V
6,HD 15337 c,7.553,644.0,2.39,54.4,8.11,6.3,17.18,82.0,7.0,0.1288,4.1,2.2,3.4,-27.6,5125.0,K1 V
7,HAT-P-11 b,7.608,829.0,4.36,57.2,23.4,17.55,4.89,127.0,31.57,0.5517,18.1,nan,2.0,48.1,4780.0,K4
8,GJ 9827 d,7.984,686.0,2.02,81.4,4.04,4.74,6.2,83.0,12.31,0.1512,5.9,1.2,2.0,-1.3,4340.0,K5 V


<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
Now we proceed to make a table of TOIs as well. This table is more uncertain of course because these systems have not been followed up in detail; but it should give us a hint of what kind of planets are in the pipe right now.

In [159]:
from astropy.table import Table
import urllib
import astropy.coordinates as coord

Load TOI's from the Exofop webpage:

In [160]:
tois = Table.read("https://exofop.ipac.caltech.edu/tess/download_toi.php?sort=toi&output=csv", format='ascii.csv')

<br/>
After this line, you should again run the accompanying script Pandexo.py to add Pandexo simulations of the SNR and observation time (minus overhead) as columns to the TOI table. This table is then read in again in the following line:

In [161]:
#We make a table to parse the exofop information in.
TOI_transiting = Table()#analogous to the 'transiting' Table above.
TOI_transiting['pl_name'] = ['TOI '+str(i) for i in tois['TOI']]#Parse the planet name
TOI_transiting['comments'] = tois['Comments']#Parse the alternative name if any.
TOI_transiting['pl_orbper'] = tois['Period (days)']*u.d#Parse orbital period.
TOI_transiting['r_earth'] = tois['Planet Radius (R_Earth)']*u.earthRad#Parse the planet radius
TOI_rp = TOI_transiting['r_earth'].to(u.earthRad)#Extract and cast to earth radii.
TOI_transiting['mass_predicted']=[mass_radius(x).value for x in TOI_rp]*u.earthMass#Fill in the MR-relationship again.
TOI_transiting['teq'] = tois['Planet Equil Temp (K)']*u.Kelvin#The equilibrium temperature, if provided.
TOI_transiting['duration']=(tois['Duration (hours)']*u.h)
TOI_transiting['Teff']=tois['Stellar Eff Temp (K)']*u.Kelvin
TOI_transiting['MT']=tois['TESS Mag']
TOI_transiting['Rstar']=tois['Stellar Radius (R_Sun)']*u.solRad
TOI_transiting['TIC ID']=tois['TIC ID']
TOI_transiting['Jmag']=-1.0#default
pickle.dump(TOI_transiting, open( "TOI_table.p", "wb" ) )

<br/>
Load the table back in, and compute some additional quantities:

In [162]:
TOI_transiting = pickle.load(open("TOI_table_pandexo.p", "rb" ))

#Now we wish to compute the stellar mass in order to compute the semi-major axis in order to compute the equilibrium temperature for those planets for which it is missing.
# lum = (4*np.pi*(tois['Stellar Radius (R_Sun)']*u.solRad)**2*const.sigma_sb*TOI_transiting['Teff']**4).to(u.solLum)
lum = (4*np.pi*(tois['Stellar Radius (R_Sun)']*u.solRad)**2*const.sigma_sb*(tois['Stellar Eff Temp (K)']*u.Kelvin)**4).to(u.solLum)
Ms = (lum**(1/3.5)).value*u.solMass
Mp = TOI_transiting['mass_predicted']
a_computed = (((tois['Period (days)']*u.d)**2*const.G*(Ms+Mp)/(4.0*np.pi**2))**(1/3)).to(u.au)
TOI_transiting['sma']=a_computed
TOI_transiting['teq_predicted'] = (TOI_transiting['Teff'] * np.sqrt(tois['Stellar Radius (R_Sun)']*u.solRad / 2 / a_computed)).decompose()
#TOI_transiting['teq'][TOI_transiting['teq'] == 0] = tois['Stellar Eff Temp (K)'][TOI_transiting['teq'] == 0]*u.Kelvin * np.sqrt(tois['Stellar Radius (R_Sun)'][TOI_transiting['teq'] == 0]*u.solRad / 2 / a_computed[TOI_transiting['teq'] == 0])


#Now on to the scale height:
TOI_gp = const.G * Mp / (tois['Planet Radius (R_Earth)']*u.earthRad)**2
TOI_transiting['H'] = ((const.k_B * TOI_transiting['teq']) / (mean_molecular_mass * TOI_gp)).to(u.km)
TOI_transiting['H'][TOI_transiting['teq']==0] = ((const.k_B * TOI_transiting['teq_predicted'][TOI_transiting['teq']==0]) / (mean_molecular_mass * TOI_gp[TOI_transiting['teq']==0])).to(u.km)
TOI_transiting['signal'] = (2*tois['Planet Radius (R_Earth)']*u.earthRad*TOI_transiting['H']/(tois['Stellar Radius (R_Sun)']*u.solRad)**2).decompose()

Apply selection criteria defined above to the TOI's as well:

In [163]:
#Now we apply selection criteria and adjust this into a more formatted table that we print to screen.
TOI_transiting['teq_condition']=TOI_transiting['teq']*1.0#This is defined so that the Teq can be replaced by the predicted one if Teq=0.
#This is used only for applying the Teq condition.
TOI_transiting['teq_condition'][TOI_transiting['teq']==0]=TOI_transiting['teq_predicted'][TOI_transiting['teq']==0]*1.0
temp_constraints = (TOI_transiting['teq_condition'] < teq_max) & (TOI_transiting['teq_condition'] > teq_min)
rad_constraints = (TOI_rp < rad_max) & (TOI_rp > rad_min)
TOI_targets = TOI_transiting[rad_constraints & temp_constraints]#This is the table that will be plotted.

<br/>
Finally, prepare for outputting a table to screen by rounding off relevant quantities:

In [164]:
TOI_targets['Teq']=np.round(TOI_targets['teq'],0)
TOI_targets['Teq (pred)']=np.round(TOI_targets['teq_predicted'],0)
TOI_targets['Teq (cond)']=np.round(TOI_targets['teq_condition'],0)
TOI_targets['Rp']=np.round(TOI_targets['r_earth'],2)
TOI_targets['H (pred)']=np.round(TOI_targets['H'],0)
TOI_targets['P']=np.round(TOI_targets['pl_orbper'],2)*u.d
TOI_targets['Dur']=np.round(TOI_targets['duration'],1)*u.h
TOI_targets['TESS Mag']=np.round(TOI_targets['MT'],2)
# targets['Dur'][TOI_transiting['Dur']==0.0]=np.nan

# targets['H_ppm']=np.round(1e6*targets['signal'],2)
# targets['JWST_ppm']=np.round(1e6*targets[JWST_mode+'_error'],1)
# targets['SNR_H']=np.round(targets['signal']/targets[JWST_mode+'_error'],4)
# targets['TSM (J)'] = np.round(targets['Kempton_metric'].value*100,1)
TOI_targets['T_eff']=np.round(TOI_targets['Teff'],0)
# targets['Mass (measured)']=np.round(targets['pl_bmasse'],2)*u.earthMass
TOI_targets['Mass (pred)']=np.round(TOI_targets['mass_predicted'],2)*u.earthMass
TOI_targets.sort('TESS Mag')
TOI_targets[['pl_name', 'comments','TESS Mag', 'Teq','Teq (pred)','Rp','Mass (pred)','P','H (pred)','Dur','T_eff']].show_in_notebook()

idx,pl_name,comments,TESS Mag,Teq,Teq (pred),Rp,Mass (pred),P,H (pred),Dur,T_eff
,,,,K,K,,earthMass,d,km,h,K
0,TOI 396.01,follow-up in progress,5.91,1146.0,1312.0,2.37,6.24,5.97,146.0,2.8,6308.0
1,TOI 1726.01,--,6.27,886.0,982.0,2.16,5.31,7.11,110.0,3.2,5694.0
2,TOI 1726.02,multi,6.27,622.0,689.0,2.64,7.46,20.55,82.0,4.0,5694.0
3,TOI 1689.01,Potentially on the neighboring star; possibly has a stellar companion,6.32,891.0,964.0,2.05,4.85,9.12,109.0,5.1,5470.0
4,TOI 554.01,Gaia DR2 R = 1.4 R_Sun; period might be twice (14.1 day).,6.44,1152.0,1428.0,3.41,11.57,7.05,164.0,3.3,6338.0
5,TOI 480.01,centroid shift may be smaller than reported,6.78,1118.0,1353.0,3.08,9.74,6.87,154.0,3.6,6213.0
6,TOI 1821.01,HD 97658 b,6.99,1020.0,767.0,2.15,5.27,9.5,126.0,2.3,5120.0
7,TOI 186.01,known TOI; second TCE at 7.78 days,6.99,358.0,363.0,3.25,10.66,66.8,50.0,3.4,4629.0
8,TOI 909.01,High priority (2.7 Rp); bright; nearby star,7.05,1386.0,1581.0,2.73,7.94,3.9,185.0,4.5,6059.0
